In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision import models
import model
import datasets
import matplotlib.pyplot as plt
import numpy as np
import math
import copy
import custom_model
from torch.optim.lr_scheduler import StepLR
import itertools

In [ ]:
# Make sure to use the GPU. The following line is just a check to see if GPU is availables
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

### Dataset Loading

In [ ]:
# Load the dataset
root = 'birds_dataset/images/'

# Small datasets used for local testing
# train_dataset = datasets.bird_dataset(root, 'birds_dataset/small_train_list.txt')
# test_dataset = datasets.bird_dataset(root, 'birds_dataset/small_test_list.txt')

# Full datasets
train_dataset = datasets.bird_dataset(root, 'birds_dataset/train_list.txt')
test_dataset = datasets.bird_dataset(root, 'birds_dataset/test_list.txt')

In [ ]:
# Split off validation sets
num_classes = 20
val_imgs_per_class = 3

# Define validation indices
val1_idxs = list(range(0, len(train_dataset), 
                       int(len(train_dataset) / num_classes / val_imgs_per_class)))
train1_idxs = list(set(list(range(len(train_dataset)))) - set(val1_idxs))
val2_idxs = list(range(val_imgs_per_class, len(train_dataset), 
                       int(len(train_dataset) / num_classes / val_imgs_per_class)))
train2_idxs = list(set(list(range(len(train_dataset)))) - set(val2_idxs))

# Define validation subsets
val1_dataset = torch.utils.data.Subset(train_dataset, val1_idxs)
train1_dataset = torch.utils.data.Subset(train_dataset, train1_idxs)
val2_dataset = torch.utils.data.Subset(train_dataset, val2_idxs)
train2_dataset = torch.utils.data.Subset(train_dataset, train2_idxs)

# Define dataloaders
train1_dataloader = DataLoader(train1_dataset, batch_size=5, shuffle=True, num_workers=2)
val1_dataloader = DataLoader(val1_dataset, batch_size=10, shuffle=True, num_workers=2)

train2_dataloader = DataLoader(train2_dataset, batch_size=5, shuffle=True, num_workers=2)
val2_dataloader = DataLoader(val2_dataset, batch_size=10, shuffle=True, num_workers=2)

test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True, num_workers=2)

### Training & Validation Functions

In [ ]:
# Weight initialization functions
# Xavier initialization
def init_weights_xavier(model):
    if type(model) == nn.Linear:
        torch.nn.init.xavier_uniform_(model.weight)
        model.bias.data.fill_(0)

In [ ]:
# Loop over batches within a single epoch
# Calculate and return loss, accuracy
def batch_loop(model, criterion, optimizer, step, step_gamma, dataloader, training=True):
    """
    model      - the neural network model being trained or evaluated
    criterion  - used to calculate the loss
    optimizer  - used to optimize during training
    step       - step size for learning rate decay. None if no decay is used
    step_gamma - gamma for learning rate decay. None if no decay is used
    dataloader - the dataset on which model is being trained or evaluated
    training   - determines whether model is being trained (True) or evaluated (False)
    """
    model.training = training
    running_loss = 0
    running_acc = 0

    # Step-wise learning rate decay
    scheduler = None
    if step is not None:
        scheduler = StepLR(optimizer, step_size=step, gamma=step_gamma)

    num_batches = 0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Ensure parameter gradients start at zero
        optimizer.zero_grad()
        
        # Forward pass
        output = model(inputs)
        
        # Calculate loss & backprop only if in train mode
        loss = criterion(output, labels.long())
        if training:
            loss.backward()
            optimizer.step()

        # Record running loss & accuracy
        running_loss += loss.item() # Current loss is average over batch
        preds = torch.argmax(output, axis=1)
        running_acc += (preds == labels).float().sum().item()
        num_batches += 1
    
    if training and step is not None:
        scheduler.step()
        
    # Return average loss and accuracy over size of dataset
    return (running_loss / num_batches), (running_acc / len(dataloader.dataset))

In [ ]:
# Trains model on train_dataloader dataset for num_epochs; evaluates on val_dataloader for each epoch.
# Returns best model & its validation loss, as well as loss & accuracy values across epochs
def train_loop(model, criterion, optimizer, step, step_gamma, train_dataloader, val_dataloader, num_epochs):
    """
    model            - the neural network model being trained
    criterion        - used to calculate the loss
    optimizer        - used to optimize during training
    step             - step size for learning rate decay. None if no decay is used
    step_gamma       - gamma for learning rate decay. None if no decay is used
    train_dataloader - the dataset on which to train the model
    val_dataloader   - the dataset on which to evaluate the model
    num_epochs       - number of training epochs
    """
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    best_model = model
    best_loss = math.inf

    for epoch in range(num_epochs):
        train_loss, train_accuracy = \
            batch_loop(model, criterion, optimizer, step, step_gamma, train_dataloader, training=True)
        val_loss, val_accuracy = \
            batch_loop(model, criterion, optimizer, step, step_gamma, val_dataloader, training=False)

        # Record epoch loss, accuracy
        print("Epoch", epoch, "train loss, acc:", train_loss, train_accuracy)
        print("Epoch", epoch, "val loss, acc:", val_loss, val_accuracy)
        train_losses.append(train_loss)
        train_accs.append(train_accuracy)
        val_losses.append(val_loss)
        val_accs.append(val_accuracy)

        # Record best model
        if val_loss < best_loss:
            best_model = copy.deepcopy(model)
            best_loss = val_loss

    print("Best:", best_loss)
    return best_model, best_loss, train_losses, train_accs, val_losses, val_accs

In [ ]:
# Train with 2-fold cross validation, using pre-loaded training & validation sets.
# Reinitializes the model between epochs to ensure a clean slate.

# For each epoch iterate over your dataloaders/datasets, pass it to your NN model, get output,
# calculate loss, and backpropagate using optimizer

# Returns final models, optimizers, criterion, as well as per-epoch loss & accuracy
def train_validation(epochs, model_init, hyperparam_init, step, step_gamma):
    """
    epochs          - number of training epochs
    model_init      - initializes and returns the model
    hyperparam_init - initializes and returns the model's criterion and optimizer
    step            - step size for learning rate decay. None if no decay is used
    step_gamma      - gamma for learning rate decay. None if no decay is used
    """
    
    # Validation set 1
    # (Re)initialize model before each training loop
    model = model_init()
    criterion, optimizer_1 = hyperparam_init(model)
    best_model_1, best_loss_1, train_losses_1, train_accs_1, val_losses_1, val_accs_1 = \
        train_loop(model, criterion, optimizer_1, step, step_gamma, 
                   train1_dataloader, val1_dataloader, num_epochs=epochs)

    # Validation set 2
    model = model_init()
    _, optimizer_2 = hyperparam_init(model)
    best_model_2, best_loss_2, train_losses_2, train_accs_2, val_losses_2, val_accs_2 = \
        train_loop(model, criterion, optimizer_2, step, step_gamma, 
                   train2_dataloader, val2_dataloader, num_epochs=epochs)
    
    
    # Record average losses and accuracies across folds
    train_losses = (np.array(train_losses_1) + np.array(train_losses_2)) / 2
    train_accs = (np.array(train_accs_1) + np.array(train_accs_2)) / 2
    val_losses = (np.array(val_losses_1) + np.array(val_losses_2)) / 2
    val_accs = (np.array(val_accs_1) + np.array(val_accs_2)) / 2
    
    # Return best models/losses/optimizers, per-epoch loss & accuracy
    optimizers = [optimizer_1, optimizer_2]
    best_models = [best_model_1, best_model_2]
    best_losses = [best_loss_1, best_loss_2]
    losses_and_accs = [train_losses, train_accs, val_losses, val_accs]
    return best_models, best_losses, optimizers, criterion, losses_and_accs

### Testing & Plotting Functions

In [ ]:
# Identify the best model, given two models and their validation losses
# Return the best model & its optimizer
def best_model(models, losses, optimizers):
    if losses[0] <= losses[1]:
        return models[0], optimizers[0]
    return models[1], optimizers[1]

In [ ]:
# Test the given model
def test_model(best_model, criterion, optimizer, step, step_gamma):
    test_loss, test_acc = batch_loop(best_model, criterion, optimizer, step, step_gamma, 
                                     test_dataloader, training=False)
    print("Test loss:", test_loss)
    print("Test accuracy:", test_acc)

In [ ]:
# Utility plotting function
def plot_results(train_results, val_results, num_epochs, model_type, result_type):
    """
    Plot results across epochs.
    """
    fig = plt.figure() 
    x = list(range(num_epochs))

    plt.plot(x, train_results, label=('Training ' + result_type))
    plt.plot(x, val_results, label=('Validation ' + result_type))

    plt.legend(loc ='upper right') 
    plt.title(model_type + ': Training and Validation ' + result_type) 
    plt.xlabel('Epoch')
    plt.ylabel(result_type)
    plt.savefig(model_type + "_" + result_type + '.png')
    plt.show()

## Models
### Baseline Model

In [ ]:
%%capture

# Create NN model object
def init_model_baseline():
    model_baseline = model.baseline_Net(classes = 20)
    model_baseline.to(device)
    return model_baseline

In [ ]:
%%capture
# Initialize hyperparameters for baseline model
def init_hyperparameters_baseline(model):
    criterion_baseline = nn.CrossEntropyLoss()
    optimizer_baseline = torch.optim.Adam(model.parameters(), lr=1e-4)
    model.apply(init_weights_xavier)
    
    return criterion_baseline, optimizer_baseline

baseline_epochs = 25

In [ ]:
%%time
# Train baseline model
base_models, base_losses, base_optimizers, base_criterion, base_losses_and_accs = \
    train_validation(baseline_epochs, init_model_baseline, init_hyperparameters_baseline, None, None)

In [ ]:
# Plot baseline model per-epoch losses & performance
base_train_losses = base_losses_and_accs[0]
base_train_accs = base_losses_and_accs[1]
base_val_losses = base_losses_and_accs[2]
base_val_accs = base_losses_and_accs[3]

plot_results(base_train_losses, base_val_losses, baseline_epochs, model_type="Baseline", result_type="Loss")
plot_results(base_train_accs, base_val_accs, baseline_epochs, model_type="Baseline", result_type="Accuracy")

In [ ]:
# Identify & test best model
baseline_nn, baseline_optimizer = \
    best_model(base_models, base_losses, base_optimizers)
test_model(baseline_nn, base_criterion, baseline_optimizer, None, None)

### Custom Model
##### Hyperparameters
- Epochs: 100
- Learning rate: 1e-4
- Learning rate decay steps: 1
- Learning rate decay gamma: 0.1
- Batch size: 5
- Activations: ReLU
- Optimizer: Adam
- Weight initialization: Xavier

In [ ]:
%%capture
# Define custom model object
def init_model_custom():
    model = custom_model.custom_Net(classes = 20)
    model.to(device)
    return model

In [ ]:
%%capture
# Initialize hyperparameters for custom model
def init_hyperparameters_custom(model):
    criterion_custom = nn.CrossEntropyLoss()
    optimizer_custom = torch.optim.Adam(model.parameters(), lr=1e-4)
    model.apply(init_weights_xavier)
    
    return criterion_custom, optimizer_custom

custom_epochs = 100
custom_step = 1
custom_step_gamma = 0.1

In [ ]:
# Train custom model
custom_models, custom_losses, custom_optimizers, custom_criterion, custom_losses_and_accs = \
    train_validation(custom_epochs, init_model_custom, init_hyperparameters_custom, custom_step, 
                     custom_step_gamma)

In [ ]:
# Plot custom model per-epoch losses & performance
custom_train_losses = custom_losses_and_accs[0]
custom_train_accs = custom_losses_and_accs[1]
custom_val_losses = custom_losses_and_accs[2]
custom_val_accs = custom_losses_and_accs[3]

plot_results(custom_train_losses, custom_val_losses, custom_epochs, model_type="Custom", result_type="Loss")
plot_results(custom_train_accs, custom_val_accs, custom_epochs, model_type="Custom", result_type="Accuracy")

In [ ]:
# Identify & test best model
custom_nn, custom_optimizer = \
    best_model(custom_models, custom_losses, custom_optimizers)
test_model(custom_nn, custom_criterion, custom_optimizer, custom_step, custom_step_gamma)

## Transfer Learning Models
### VGG-16: Pre-Trained Model Loading

In [ ]:
# Import pretrained vgg-16 model
transfer_model_vgg = models.vgg16(pretrained=True)

### VGG-16: Fixed Feature Extractor
##### Hyperparameters
- Epochs: 100
- Learning rate: 1e-4
- Learning rate decay steps: 1
- Learning rate decay gamma: 0.1
- Batch size: 5
- Optimizer: Adam

In [ ]:
# Initialize frozen vgg model, with last fc layer replaced
def init_model_vgg_fixed():
    model_vgg = copy.deepcopy(transfer_model_vgg)
    
    # Freeze existing parameters
    for param in model_vgg.parameters():
        param.requires_grad = False

    model_vgg.classifier[6] = torch.nn.Linear(model_vgg.classifier[6].in_features, num_classes)
    model_vgg.to(device)
    return model_vgg

In [ ]:
# Initialize hyperparameters for fixed vgg model
def init_hyperparams_vgg_fixed(model_vgg):
    criterion_vgg = nn.CrossEntropyLoss()
    optimizer_vgg = torch.optim.Adam(model_vgg.parameters(), lr=1e-4)
    return criterion_vgg, optimizer_vgg

vgg_fixed_epochs = 100
vgg_fixed_step = 1
vgg_fixed_step_gamma = 0.1

In [ ]:
# Train fixed vgg model
vgg_fixed_models, vgg_fixed_losses, vgg_fixed_optimizers, \
    vgg_fixed_criterion, vgg_fixed_losses_and_accs = \
        train_validation(vgg_fixed_epochs, init_model_vgg_fixed, init_hyperparams_vgg_fixed, 
                         vgg_fixed_step, vgg_fixed_step_gamma)

In [ ]:
# Plot fixed vgg model per-epoch losses & performance
vgg_fixed_train_losses = vgg_fixed_losses_and_accs[0]
vgg_fixed_train_accs = vgg_fixed_losses_and_accs[1]
vgg_fixed_val_losses = vgg_fixed_losses_and_accs[2]
vgg_fixed_val_accs = vgg_fixed_losses_and_accs[3]

plot_results(vgg_fixed_train_losses, vgg_fixed_val_losses, vgg_fixed_epochs, model_type="Custom",
             result_type="Loss")
plot_results(vgg_fixed_train_accs, vgg_fixed_val_accs, vgg_fixed_epochs, model_type="Custom",
             result_type="Accuracy")

In [ ]:
# Identify & test best model
vgg_fixed_nn, vgg_fixed_optimizer = \
    best_model(vgg_fixed_models, vgg_fixed_losses, vgg_fixed_optimizers)
test_model(vgg_fixed_nn, vgg_fixed_criterion, vgg_fixed_optimizer, vgg_fixed_step, vgg_fixed_step_gamma)

### VGG-16 Finetuned Model
##### Hyperparameters
- Epochs: 100
- Learning rate: 1e-5
- Learning rate decay steps: 1
- Learning rate decay gamma: 0.1
- Batch size: 5
- Optimizer: Adam

In [ ]:
# Initialize fineturned vgg model, with last fc layer replaced
def init_model_vgg_tuned():
    model_vgg = copy.deepcopy(transfer_model_vgg)
    
    model_vgg.classifier[6] = torch.nn.Linear(model_vgg.classifier[6].in_features, num_classes)
    model_vgg.to(device)
    return model_vgg

In [ ]:
# Initialize hyperparameters for finetuned vgg model
def init_hyperparams_vgg_tuned(model_vgg):
    criterion_vgg = nn.CrossEntropyLoss()
    optimizer_vgg = torch.optim.Adam(model_vgg.parameters(), lr=1e-5)
    return criterion_vgg, optimizer_vgg

vgg_tuned_epochs = 25
vgg_tuned_step = 1
vgg_tuned_step_gamma = 0.1

In [ ]:
# Train finetuned vgg model
vgg_tuned_models, vgg_tuned_losses, vgg_tuned_optimizers, \
    vgg_tuned_criterion, vgg_tuned_losses_and_accs = \
        train_validation(vgg_tuned_epochs, init_model_vgg_tuned, init_hyperparams_vgg_tuned, 
                         vgg_tuned_step, vgg_tuned_step_gamma)

In [ ]:
# Plot tuned vgg model per-epoch losses & performance
vgg_tuned_train_losses = vgg_tuned_losses_and_accs[0]
vgg_tuned_train_accs = vgg_tuned_losses_and_accs[1]
vgg_tuned_val_losses = vgg_tuned_losses_and_accs[2]
vgg_tuned_val_accs = vgg_tuned_losses_and_accs[3]

plot_results(vgg_tuned_train_losses, vgg_tuned_val_losses, vgg_tuned_epochs, model_type="VGG-Tuned", \
             result_type="Loss")
plot_results(vgg_tuned_train_accs, vgg_tuned_val_accs, vgg_tuned_epochs, model_type="VGG-Tuned", \
             result_type="Accuracy")

In [ ]:
# Identify & test best model
vgg_tuned_nn, vgg_tuned_optimizer = \
    best_model(vgg_tuned_models, vgg_tuned_losses, vgg_tuned_optimizers)
test_model(vgg_tuned_nn, vgg_tuned_criterion, vgg_tuned_optimizer, vgg_tuned_step, vgg_tuned_step_gamma)

### ResNet 18: Pre-Trained Model Loading

In [ ]:
# Import pretrained resnet 18 model
transfer_model_resnet = models.resnet18(pretrained=True)

### ResNet 18: Fixed Feature Extractor
##### Hyperparameters
- Epochs: 100
- Learning rate: 1e-4
- Learning rate decay steps: 1
- Learning rate decay gamma: 0.9
- Batch size: 5
- Optimizer: Adam

In [ ]:
# Initialize frozen resnet model, with last fc layer replaced
def init_model_resnet_fixed():
    model_resnet = copy.deepcopy(transfer_model_resnet)
    # Freeze existing parameters
    for param in model_resnet.parameters():
        param.requires_grad = False

    model_resnet.fc = torch.nn.Linear(model_resnet.fc.in_features, num_classes)
    model_resnet.to(device)
    return model_resnet

In [ ]:
# Initialize hyperparameters for fixed resnet model
def init_hyperparams_resnet_fixed(model_resnet):
    criterion_resnet = nn.CrossEntropyLoss()
    optimizer_resnet = torch.optim.Adam(model_resnet.parameters(), lr=1e-4)
    return criterion_resnet, optimizer_resnet

resnet_fixed_epochs = 100
resnet_fixed_step = 1
resnet_fixed_step_gamma = 0.9

In [ ]:
# Train fixed resnet model
resnet_fixed_models, resnet_fixed_losses, resnet_fixed_optimizers, \
    resnet_fixed_criterion, resnet_fixed_losses_and_accs = \
        train_validation(resnet_fixed_epochs, init_model_resnet_fixed, init_hyperparams_resnet_fixed, 
                         resnet_fixed_step, resnet_fixed_step_gamma)

In [ ]:
# Plot fixed resnet model per-epoch losses & performance
resnet_fixed_train_losses = resnet_fixed_losses_and_accs[0]
resnet_fixed_train_accs = resnet_fixed_losses_and_accs[1]
resnet_fixed_val_losses = resnet_fixed_losses_and_accs[2]
resnet_fixed_val_accs = resnet_fixed_losses_and_accs[3]

plot_results(resnet_fixed_train_losses, resnet_fixed_val_losses, resnet_fixed_epochs, model_type="Resnet-Fixed",
             result_type="Loss")
plot_results(resnet_fixed_train_accs, resnet_fixed_val_accs, resnet_fixed_epochs, model_type="Resnet-Fixed",
             result_type="Accuracy")

In [ ]:
# Identify & test best model
resnet_fixed_nn, resnet_fixed_optimizer = \
    best_model(resnet_fixed_models, resnet_fixed_losses, resnet_fixed_optimizers)

test_model(resnet_fixed_nn, resnet_fixed_criterion, resnet_fixed_optimizer, resnet_fixed_step, 
           resnet_fixed_step_gamma)

### ResNet 18: Finetuned Model
##### Hyperparameters
- Epochs: 25
- Learning rate: 1e-4
- Learning rate decay steps: 1
- Learning rate decay gamma: 0.9
- Batch size: 5
- Optimizer: Adam

In [ ]:
# Initialize frozen resnet model, with last fc layer replaced
def init_model_resnet_tuned():
    model_resnet = copy.deepcopy(transfer_model_resnet)

    model_resnet.fc = torch.nn.Linear(model_resnet.fc.in_features, num_classes)
    model_resnet.to(device)
    return model_resnet

In [ ]:
# Initialize hyperparameters for fixed resnet model
def init_hyperparams_resnet_tuned(model_resnet):
    criterion_resnet = nn.CrossEntropyLoss()
    optimizer_resnet = torch.optim.Adam(model_resnet.parameters(), lr=1e-4)
    return criterion_resnet, optimizer_resnet

resnet_tuned_epochs = 25
resnet_tuned_step = 1
resnet_tuned_step_gamma = 0.9

In [ ]:
# Train fixed resnet model
resnet_tuned_models, resnet_tuned_losses, resnet_tuned_optimizers, resnet_tuned_criterion, \
    resnet_tuned_losses_and_accs = \
        train_validation(resnet_tuned_epochs, init_model_resnet_tuned, init_hyperparams_resnet_tuned, 
                         resnet_tuned_step, resnet_tuned_step_gamma)

In [ ]:
# Plot fixed resnet model per-epoch losses & performance
resnet_tuned_train_losses = resnet_tuned_losses_and_accs[0]
resnet_tuned_train_accs = resnet_tuned_losses_and_accs[1]
resnet_tuned_val_losses = resnet_tuned_losses_and_accs[2]
resnet_tuned_val_accs = resnet_tuned_losses_and_accs[3]

plot_results(resnet_tuned_train_losses, resnet_tuned_val_losses, resnet_tuned_epochs, model_type="Resnet-Tuned", \
             result_type="Loss")
plot_results(resnet_tuned_train_accs, resnet_tuned_val_accs, resnet_tuned_epochs, model_type="Resnet-Tuned", \
             result_type="Accuracy")

In [ ]:
# Identify & test best model
resnet_tuned_nn, resnet_tuned_optimizer = \
    best_model(resnet_tuned_models, resnet_tuned_losses, resnet_tuned_optimizers)
test_model(resnet_tuned_nn, resnet_tuned_criterion, resnet_tuned_optimizer, resnet_tuned_step, 
           resnet_tuned_step_gamma)

## Weight and Feature Maps
### Plotting & Extraction Functions

In [ ]:
# Plot the given map in a square grid
def plot_maps(data, data_type, title):
    ncols = int(math.sqrt(data.size(0))) + 1
    nrows = 1 + (data.size(0) - 1)//ncols

    # figure width and height adjusted to # of plots
    figwidth = ncols*0.6+(ncols-1)*0.1+0.6
    figheight = nrows*0.6+(nrows-1)*0.1+0.6
    top = (1 - 0.2/math.sqrt(figheight))

    fig, plots = plt.subplots(nrows, ncols, figsize=(12,12))
    fig.set_size_inches(figwidth,figheight)
    fig.suptitle(title, fontsize=16)
    fig.subplots_adjust(top=top)

    for idx in range(data.size(0)):
        if data_type == "Weight": # if weight map, get first channel only
            datum = data[idx, 0].squeeze().cpu()
        else: # if "Feature", get all channels
            datum = data[idx].squeeze().cpu()
            
        r = idx // ncols
        c = idx % ncols
        plots[r,c].imshow(datum)
        plots[r,c].axis('off')
        plots[r,c].set_xticklabels([])
        plots[r,c].set_yticklabels([])

In [ ]:
# Plot feature maps for layers named in layersToPlot
def get_feature_maps(model, layersToPlot, model_type):
    first_test_data = test_dataset[0][0].unsqueeze(0).to(device)
    
    activation = {}
    # hook according to pytorch documentation
    def get_activation(name):
        def hook(layer, m_input, m_output):
            activation[name] = m_output.detach()
        return hook
    
    # get activation outputs
    for name, layer in model._modules.items():
        layer.register_forward_hook(get_activation(name))
    output = model(first_test_data)

    # plot feature maps for the given layers 
    for layer in layersToPlot:
        act = activation[layer].squeeze() # "layer output shape" = size = [D2, W2, H2]
        plot_maps(act, "Feature", model_type + ": Layer " + layer)

In [ ]:
# Plot weight map of given layer
def get_weight_maps(layer, model_type):
    filters = layer.weight.detach() # "filter shape" = size = [64,3,3,3]
    plot_maps(filters, "Weight", model_type + " Weights") 

### Custom Model

In [ ]:
# Plot custom model feature & weight maps
get_feature_maps(custom_nn, ['b1', 'b3', 'b6'], "Custom")
get_weight_maps(custom_nn.b1[0], "Custom")

### VGG Models

In [ ]:
# Plot fixed vgg model feature & weight maps
get_feature_maps(vgg_fixed_nn.features, ['0', '14', '28'], "VGG 16 (Fixed)")
get_weight_maps(vgg_fixed_nn.features[0], "VGG 16 (Fixed)")

In [ ]:
# Plot finetuned vgg model feature & weight maps
get_feature_maps(vgg_tuned_nn.features, ['0', '14', '28'], "VGG 16 (Finetuned)")
get_weight_maps(vgg_tuned_nn.features[0], "VGG 16 (Finetuned)")

### ResNet Models

In [ ]:
# Plot fixed resnet model feature & weight maps
get_feature_maps(resnet_fixed_nn, ['conv1', 'layer2', 'layer4'], "ResNet 18 (Fixed)")
get_weight_maps(resnet_fixed_nn.conv1, "ResNet 18 (Fixed)")

In [ ]:
# Plot finetuned resnet model feature & weight maps
get_feature_maps(resnet_tuned_nn, ['conv1', 'layer2', 'layer4'], "ResNet 18 (Finetuned)")
get_weight_maps(resnet_tuned_nn.conv1, "ResNet 18 (Finetuned)")